In [1]:
import pandas as pd
import json
import os

In [2]:
members=[]

with open('conversation.json') as json_file:
    data = json.load(json_file)
    members = data['members']
print(members[0])

{'user_id': '27676046', 'nickname': '🎅🏿', 'image_url': 'https://i.groupme.com/434x434.jpeg.a4ef57346b714c3d922dcdd99e8304cf', 'id': '432140249', 'muted': False, 'autokicked': False, 'roles': ['user'], 'name': 'Dylan Fox'}


In [3]:
user_ids = list(map(lambda user_object: (user_object["user_id"]), members))
user_names = list(map(lambda user_object: user_object["name"], members))

joined = list(zip(user_ids, user_names))

df = pd.DataFrame(joined, columns = ['id', 'name'])
df['num_sent'] = 0
df['likes_per_message'] = 0
df['likes_given'] = 0
df['percentage_eta'] = 0
df.head()

,id,name,num_sent,likes_per_message,likes_given,percentage_eta
0,27676046,Dylan Fox,0,0,0,0
1,30017328,Sam Schoberg,0,0,0,0
2,24280778,Antonio Karides,0,0,0,0
3,36556659,Nolan McNeill,0,0,0,0
4,39092107,Jack Singelstad,0,0,0,0


In [4]:
num_sent = {}
likes_recieved = {}
likes_given = {}

for i in df['id']:
    num_sent[i] = 0
    likes_recieved[i] = 0
    likes_given[i] = 0

In [5]:
with open('message.json') as json_file:
    data = json.load(json_file)
    
    for mes in data:
        if(mes['sender_id'] in user_ids):
            num_sent[mes['sender_id']] = num_sent[mes['sender_id']] + 1
            likes_recieved[mes['sender_id']] = likes_recieved[mes['user_id']] + len(mes['favorited_by'])
            
            for liker in mes['favorited_by']:
                if liker in user_ids:
                    likes_given[liker] = likes_given[liker] + 1

In [6]:
for k,v in num_sent.items():
    df.loc[df['id'] == k, "num_sent"] = v
    df.loc[df['id'] == k, "percentage_eta"] = v/len(data)*100
    
for k,v in likes_recieved.items():
    if(num_sent[k] is 0): 
        df.loc[df['id'] == k, "likes_per_message"] = -1
    else:
        df.loc[df['id'] == k, "likes_per_message"] = v/num_sent[k] 

    
for k,v in likes_given.items():
    df.loc[df['id'] == k, "likes_given"] = v


In [7]:
df.head()

,id,name,num_sent,likes_per_message,likes_given,percentage_eta
0,27676046,Dylan Fox,1321,1.745647,658,6.456500
1,30017328,Sam Schoberg,449,3.848552,726,2.194526
2,24280778,Antonio Karides,519,3.148362,1571,2.536657
3,36556659,Nolan McNeill,180,5.200000,216,0.879765
4,39092107,Jack Singelstad,1199,2.613011,1739,5.860215


In [8]:
df.loc[df['id'] == '27676046', "num_sent"][0]


1321

In [9]:
df.to_json('data.json', orient='records')

In [12]:
out = {
    "eta-lytics":[]
}

lst = list(df.iterrows())
len(lst)

for row in df.iterrows():
    if(len(out['eta-lytics']) is 25):
        
        with open('eta-lytics.json', 'w') as fp:
            json.dump(out, fp)
        
        resp = os.system("aws dynamodb batch-write-item --request-items file://eta-lytics.json")
        print(resp)
        out['eta-lytics'] = []
        
    out['eta-lytics'].append(put_request(row))


with open('eta-lytics.json', 'w') as fp:
    json.dump(out, fp)

resp = os.system("aws dynamodb batch-write-item --request-items file://eta-lytics.json")
print(resp)

0
0
0
0


In [11]:
def put_request(row):
    return (
    {
        "PutRequest": {
            "Item": {
                "name": {"S": row[1]['name']},
                'id': {"S": row[1].id},
                'messages': {"S": str(row[1].num_sent)},
                'likes_per_message': {"S": str(round(row[1].likes_per_message,2))},
                "likes_given": {"S": str(row[1].likes_given)} ,
                'percentage_eta': {"S": str(round(row[1].percentage_eta,2))}
                
            }
        }
    }
    )